In [1]:
from linkedin_api import Linkedin
import pandas as pd
import time
import json

In [2]:
username = ''
password = ''

# Authenticate using any Linkedin user account credentials
api = Linkedin(username, password)

In [4]:
input_dir = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\3. Filtered preprocessed data with country\\"
save_path_with_industry = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\4. Filtered preprocessed data with industry\\"

In [5]:
company_list = pd.read_csv(f"{save_path_with_industry}industries_not_found_batch_3.csv")
company_list = list(company_list['Company ID'])
company_list[:1]

['2692410']

In [6]:
len(company_list)

8459

In [7]:
_chunks = [company_list[i:i + 1000] for i in range(0, len(company_list), 1000)]
len(_chunks[0]), len(_chunks)

(1000, 9)

In [8]:
# Function to extract company ID and LinkedIn link
def extract_company_info(company_detail):
    try:
        company_data = json.loads(company_detail.replace("'", '"'))
        if 'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany' in company_data:
            company_info = company_data['com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany']
        elif 'com.linkedin.voyager.jobs.JobPostingCompanyName' in company_data:
            company_info = company_data['com.linkedin.voyager.jobs.JobPostingCompanyName']
        # print(company_info['company'])
        if 'company' in company_info:
            company_id = company_info['company'].split(':')[-1]
        elif 'companyResolutionResult' in company_info:
            company_id = company_info['companyResolutionResult']['entityUrn'].split(':')[-1]
        elif 'companyName' in company_info:
            company_id = company_info['companyName']
        return company_id
    except (KeyError, ValueError, TypeError) as e:
        return None, None

In [12]:
for i in range(1,6):
    print(i)
    company_l = _chunks[i]
    company_data = []
    for l in company_l:
        company_id = l
        try:
            company = api.get_company(company_id)
            industry = company['companyIndustries'][0]['localizedName']
            company_data.append({"Company ID": company_id, "Industry": industry})
        except:
            company_data.append({"Company ID": company_id, "Industry": "not found"})
    pd.DataFrame(company_data).to_csv(f"{save_path_with_industry}industries_{i}.csv")

1
2
3
4
5
